In [1]:
import numpy as np
from matplotlib import pyplot as plt

# Pre Exercise

In the google doc sheet we have 41 restaurant and 44 people. There are about 1000 rating in total. We want to do collaborative filtering with 7 hidden features.

https://docs.google.com/spreadsheets/d/143AcuFLKRLkz7yoKiEas0rABfuKPYCSlVKY1DUbFiBo/edit#gid=0

1) What is the size of matrix R?

In [6]:
# 41 columns , 7 rows

2) What is the size of matrix P?

In [7]:
# 44 columns, 7 rows

3) What should be the dimension of the rating matrix T? Write a numpy statement to calculate the $T$ from $R$ and $P$.

In [8]:
# R.T P --> 
# 41 row 44 columns

4)Import the google sheet. (export it to csv and read line by line). The google sheet and the notes has a bit different conventtion for rating matrix so be careful.

In [6]:
def read_rating():
    with open('rating.csv', encoding='utf-8') as f:
        iline = 0
        lines = f.readlines()
        useful_lines = lines[3:]

        names = lines[2].split(',')[2:]
        names = list(map(lambda x: x.strip(), names))

        all_ratings = []
        all_defined = []
        rnames = []
        for iline, line in enumerate(useful_lines):
            tokens = line.split(',')
            tokens = list(map(lambda x: x.strip(), tokens))
            rname = tokens[1]
            ratings = tokens[2:]
            defined = list(map(lambda x: 0 if x=='' or x=='"' else 1, ratings ))
            def clean_cast(x):
                # print x
                return 0 if x=='' or x=='"' else float(x)
            ratings = list(map(lambda x: clean_cast(x), ratings))
            all_ratings.append(ratings)
            all_defined.append(defined)
            rnames.append(rname)
        #print all_ratings, all_defined , rnames
        T = np.array(all_ratings)
        H = np.array(all_defined)
    return T, H, names, rnames
T, H, names, rnames = read_rating()

5) What is the dimension of rating matrix? AFTER you figure out, check it with rating.shape.

In [7]:
print(T.shape)

(41, 44)


6) What is the dimension of has_rating matrix? AFTER you figure out, check it with rating.shape.

In [8]:
print(H.shape)

(41, 44)


7)The update rule for $P$ is


$P = P - \eta\; dP$

Write down the formula for $dP$. Then, write a numpy code to calculate $dP$.

8)The update rule for $R$ is

$R = R- \eta\; dR$

Write down the formula for dR. Then, write a numpy code to calculate dR.

9) Write the numpy code to calculate the cost. (you may find np.sum useful)

# Exercise
You have given CSV file for all the rating. Now your job is to figure out what is the next restaurant you should go next using collaborative filtering with gradient descent.

The idea is to keep update matrix F and P with the update rule we find above many many times.

Debugging this is quite a pain. One way to do it is to monitor the value of the cost function making sure it is going in the right direction.

If you are doing it right this should not be more than 50 lines.

The convergence is quite slow. So try it 10k + times.

In [34]:
np.random.seed(555)

R = np.random.randn(4, 41)
R.shape

P = np.random.randn(4, 44)
P.shape

def cost(R, P):
    return np.sum(H*(T - R.T @ P)**2)

def find_R_P(initial_R, initial_P, n, lmb=1e-3):
    R = initial_R.copy()
    P = initial_P.copy()
    
    for i in range(n):
        tmp_P = P + 2*lmb*R@(H*(T-R.T@P))
        tmp_R = R + 2*lmb*P@(H*(T-R.T@P)).T
        R = tmp_R
        P = tmp_P
        # print(cost(R, P))
    return R, P

R, P = find_R_P(R, P, 100000)


print(cost(R,P))

228.6728180411126


# Debugging Tip
To compare your guess rating(G*has_rating) and the actual rating graphically use

plt.matshow(rating, vmin=0, vmax=5)

 if they look almost the same then your algorithm is doing the right thing 

Once you have F and P and guess matrix.
I wrote this so that visualizing your result is easier.

In [35]:
from IPython.display import HTML

class TableCell:
    
    def __init__(self, text, tc=None, color=None):
        self.text = text
        self.tc = tc
        self.color = color
    
    def to_html(self):
        return '<td>%s</td>'%self.text

#the rating and guess matrix has different convention from the notes so be sure to transpose it first
def maketable(rating, has_rating, guess, restaurants, names):
    n_rests = len(restaurants)
    n_names = len(names)
    tab = np.empty((n_rests+1, n_names+1),dtype='object')
    
    for irest in range(n_rests):
        tab[irest+1,0] = restaurants[irest]

    for iname in range(n_names):
        tab[0,iname+1] = names[iname]

    for irest in range(n_rests):
        for iname in range(n_names):
            if not has_rating[iname, irest]:
                tab[irest+1, iname+1] = TableCell('<span style="color:red">%3.2f</span>'%(guess[iname, irest]))
            else:
                tab[irest+1, iname+1] = TableCell('<span style="color:blue">%3.2f</span><span style="color:red">(%3.2f)</span>'%(rating[iname, irest], guess[iname, irest]))
    #now convert tab array to nice html table
    nrow, ncol = tab.shape
    t = []
    t.append('<table>')
    for irow in range(nrow):
        t.append('<tr>')
        for icol in range(ncol):
            cell = tab[irow,icol]
            if cell is not None:
                if isinstance(cell,TableCell):
                    t.append(tab[irow, icol].to_html())
                else:
                    t.append('<td>')
                    t.append(tab[irow, icol])
                    t.append('</td>')
            else:
                t.append('<td></td>')
        t.append('</tr>')  
    t.append('</table>')
    return '\n'.join(t)

In [36]:
# call it like this

HTML(maketable(T.T, H.T, (R.T @ P).T, rnames, names))

,Piti,Meena,Pitoon,Sohum,Sam,Keng,O,Yok,Kitty,Time,Robroo,Peem,Chanon,Ohm,Sun,Tii,Tow,John,Mint,Opal,Kelly,Jay,Sharik,Ou,PJ,Punawit,Martin,Ploy,Majeed,Bossy,Sea,Pan,Ice,Karn,May,Rin,Peter,Ham,Benz,Billy,Kanat,Sam,Boss,Best
Mai-tok-mai-tak,5.36,3.33,2.00(2.52),3.95,3.93,2.00(3.20),2.00(3.67),3.00(3.54),3.93,3.00(3.50),3.00(2.63),5.00(3.70),2.83,3.00(3.09),2.00(2.48),3.00(2.77),3.00(3.48),2.65,2.00(1.69),3.00(2.73),3.00(2.62),3.00(3.01),3.63,2.00(2.80),1.67,3.46,3.01,4.00(3.16),3.56,3.00(2.71),4.00(3.68),3.55,2.84,3.00(3.37),4.00(3.12),3.08,3.48,3.13,4.00(3.81),4.00(4.22),4.00(3.73),3.30,4.00(3.99),5.00(3.74)
Puttharaksa,4.00(3.72),3.38,4.00(2.26),4.00(3.82),4.00(4.12),3.00(3.53),4.00(3.68),3.00(3.63),3.68,3.00(3.33),2.85,4.00(3.80),3.00(2.89),4.00(3.23),3.00(3.03),4.00(4.13),4.00(3.79),3.00(3.05),3.00(2.94),3.00(3.29),3.00(3.19),3.00(3.17),3.00(3.40),3.00(2.88),2.09,3.28,3.24,3.00(3.16),3.00(3.33),2.79,4.00(3.78),4.00(3.33),4.00(3.71),3.33,4.00(3.27),3.00(3.13),3.14,3.35,1.00(3.30),4.00(4.16),3.00(3.35),3.68,4.00(3.76),4.00(3.31)
Big Mamma,2.92,4.00(4.01),0.55,5.00(4.84),5.00(5.07),6.90,5.21,4.72,4.00(4.11),2.78,4.02,7.25,2.92,9.25,6.81,23.84,6.81,10.12,11.54,7.03,6.21,3.67,0.62,3.37,17.48,5.93,5.13,2.49,3.61,2.13,4.78,-1.43,8.24,3.15,4.00(3.86),4.00(4.11),-0.17,3.00(3.01),0.86,3.88,3.00,6.96,2.16,0.48
Seefah,9.24,2.00(3.14),4.00(3.67),4.00(4.27),5.00(4.64),2.74,4.00(3.57),3.00(3.46),4.43,5.00(3.88),1.44,4.00(3.69),3.00(3.03),4.07,1.56,4.00(4.17),2.63,3.00(2.75),-0.23,1.79,1.97,2.88,4.00(4.33),2.00(2.73),0.86,4.00(3.77),2.79,3.00(3.21),4.50,2.00(2.02),3.51,3.37,1.09,3.28,3.00(2.56),1.97,3.00(3.07),4.00(3.16),4.97,4.00(4.90),4.28,2.03,5.00(4.41),4.00(4.44)
Music Square,2.00(2.32),3.82,0.90,4.09,3.00(2.97),4.00(4.36),4.33,4.00(3.93),4.00(3.85),4.00(3.18),4.00(4.40),4.62,2.57,3.19,4.00(4.05),4.00(4.00),5.00(5.15),3.86,5.00(4.86),4.30,4.00(3.55),3.00(3.19),2.33,2.00(3.01),6.42,5.00(4.16),3.58,3.00(3.02),2.77,3.51,4.00(4.16),2.97,5.32,4.00(3.61),5.00(3.98),4.00(4.98),3.83,2.72,2.41,2.00(3.40),4.00(3.56),5.55,4.00(3.43),3.00(2.75)
Mamma Mia,3.00(3.25),4.00(4.02),2.00,5.00(4.42),5.00(4.33),4.43,4.00(4.44),4.00(4.27),4.18,4.00(3.69),5.00(3.89),5.00(4.67),3.00(3.19),5.00(3.72),3.00(4.00),5.00(5.11),4.00(4.92),3.91,4.00(4.44),4.00(4.29),4.00(3.94),4.00(3.67),3.41,3.00(3.36),4.14,3.00(4.01),4.00(3.89),3.00(3.54),3.52,4.00(3.47),5.00(4.48),3.00(3.59),5.00(5.08),4.00(3.89),5.00(4.03),5.00(4.27),3.73,3.00(3.66),3.00(3.28),5.00(4.44),3.00(3.79),5.00,4.00(4.09),3.44
Srijan,4.00(4.29),3.05,3.00(2.55),3.52,3.92,3.00(2.83),3.25,5.00(3.26),3.49,3.00(3.25),2.00(2.30),4.00(3.16),2.75,2.00(2.53),2.00(2.24),2.00(1.93),2.00(3.04),2.07,1.00(1.42),2.48,2.54,2.90,3.65,3.00(2.64),-0.02,4.00(2.82),2.78,3.05,3.00(3.27),2.00(2.56),3.39,4.00(3.64),2.63,3.13,3.00(2.88),2.59,3.23,3.00(3.21),3.00(3.60),4.00(4.08),3.00(3.27),2.82,4.00(3.79),3.00(3.57)
Steak House,12.51,3.00(2.37),4.35,4.00(3.85),4.00(4.03),1.22,2.76,2.61,4.00(4.32),3.77,-0.00,2.61,2.59,3.29,-0.39,0.72,0.92,1.00(1.14),-3.95,-0.23,0.00(0.18),2.07,5.00(4.68),3.00(2.08),-1.93,3.00(3.32),1.71,2.80,4.67,1.18,2.61,3.00(3.45),-1.82,2.74,1.52,0.72,3.00(3.05),2.00(2.42),5.83,4.67,4.47,0.03,4.43,5.02
Anya,1.00(0.53),4.00(4.06),4.00(3.02),4.00(3.97),5.00(5.47),4.00(3.81),4.00(3.91),4.00(4.31),4.00(3.71),4.00(3.97),3.00(3.52),3.00(3.46),4.00(3.83),1.00(1.85),4.00(3.61),0.56,4.20,2.00(1.81),4.00(3.30),4.00(3.98),4.00(4.30),5.00(4.23),5.00(4.97),4.00(3.64),-3.41,2.35,4.00(3.86),4.00(4.27),3.00(3.47),4.00(3.94),4.55,5.71,5.00(4.96),4.15,4.17,3.44,4.34,5.00(5.10),4.00(3.88),5.00(5.29),3.00(3.27),4.05,5.00(4.82),4.00(4.27)
Chicky Chic,2.00(2.44),3.00(3.09),2.00(0.46),2.00(3.19),0.00(0.93),2.00(2.63),5.00(3.38),2.00(2.87),3.00(3.23),2.00(2.73),5.00(4.08),4.00(3.01),2.00(1.71),1.00(0.29),2.00(2.21),-4.95,3.80,0.00(0.89),2.00(1.27),2.00(2.40),0.00(1.52),2.00(2.21),2.00(2.15),3.00(2.23),3.00(2.78),3.00(3.21),3.00(2.24),2.00(2.50),1.85,3.00(3.49),3.00(3.15),3.87,3.00(3.13),3.00(3.15),3.00(3.36),5.00(4.98),5.00(4.99),2.0